<a href="https://colab.research.google.com/github/alexunderch/Bachelor_materials/blob/main/ml-mipt_advanced/DL_Research/Advanced%20Policy%20gradient/PPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import sys, os
if 'google.colab' in sys.modules and not os.path.exists('.setup_complete'):
    !wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/master/setup_colab.sh -O- | bash
    
    !wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/master/week06_policy_based/runners.py
    !wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/master/week09_policy_II/mujoco_wrappers.py
    
    !touch .setup_complete

# This code creates a virtual display to draw game images on.
# It will have no effect if your machine has a monitor.
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY")) == 0:
    !bash ../xvfb start
    os.environ['DISPLAY'] = ':1'

Selecting previously unselected package xvfb.
(Reading database ... 155062 files and directories currently installed.)
Preparing to unpack .../xvfb_2%3a1.19.6-1ubuntu4.9_amd64.deb ...
Unpacking xvfb (2:1.19.6-1ubuntu4.9) ...
Setting up xvfb (2:1.19.6-1ubuntu4.9) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Starting virtual X frame buffer: Xvfb.


# Implementing Proximal Policy Optimization 


In this notebook you will be implementing Proximal Policy Optimization algorithm, 
scaled up version of which was used to train [OpenAI Five](https://openai.com/blog/openai-five/) 
to [win](https://openai.com/blog/how-to-train-your-openai-five/) against the
world champions in Dota 2.
You will be solving a continuous control environment on which it may be easier and faster 
to train an agent, however note that PPO here may not be the best algorithm as, for example,
Deep Deterministic Policy Gradient and Soft Actor Critic may be more suited 
for continuous control environments. To run the environment you will need to install 
[pybullet-gym](https://github.com/benelot/pybullet-gym) which unlike MuJoCo 
does not require you to have a license.

To install the library:

In [3]:
import os
!git clone https://github.com/benelot/pybullet-gym.git
os.chdir('/content/pybullet-gym')
!pip install -e .

Cloning into 'pybullet-gym'...
remote: Enumerating objects: 804, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 804 (delta 21), reused 28 (delta 6), pack-reused 750
Receiving objects: 100% (804/804), 19.31 MiB | 22.37 MiB/s, done.
Resolving deltas: 100% (437/437), done.
Obtaining file:///content/pybullet-gym
     |████████████████████████████████| 89.3 MB 35 kB/s 
  Running setup.py develop for pybulletgym


The overall structure of the code is similar to the one in the A2C optional homework, but don't worry if you haven't done it, it should be relatively easy to figure it out. 
First, we will create an instance of the environment. 
We will normalize the observations and rewards, but before that you will need a wrapper that will 
write summaries, mainly, the total reward during an episode. You can either use one for `TensorFlow` 
implemented in `atari_wrappers.py` file from the optional A2C homework, or implement your own. 

In [4]:
import gym 
import pybulletgym

env = gym.make("HalfCheetahMuJoCoEnv-v0")
print("observation space: ", env.observation_space,
      "\nobservations:", env.reset())
print("action space: ", env.action_space, 
      "\naction_sample: ", env.action_space.sample())

WalkerBase::__init__
observation space:  Box(-inf, inf, (17,), float32) 
observations: [ 0.09770291 -0.01340525  0.00735227  0.07979384 -0.07401946  0.01202126
 -0.08142443  0.05965538  0.          0.          0.          0.
  0.          0.          0.          0.          0.        ]
action space:  Box(-1.0, 1.0, (6,), float32) 
action_sample:  [ 0.76114476 -0.4763767   0.18514137  0.590572    0.18258515 -0.4306853 ]


/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [5]:
import torch
import numpy as np
from collections import deque
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [6]:
def torch_steps():
    try: stepval = optimizer.state[optimizer.param_groups[0]["params"][-1]]
    except NameError: stepval = 0
    return stepval

class Summaries(gym.Wrapper):
    """ Env summaries writer base."""

    def __init__(self, env, prefix=None, running_mean_size=100, step_var =None):
        super().__init__(env)
        self.writer = SummaryWriter()
        self.episode_counter = 0
        self.prefix = prefix or self.env.spec.id
        self.step_count =0
        self.step_var = (step_var if step_var is not None
                         else torch_steps())

        nenvs = getattr(self.env.unwrapped, "nenvs", 1)
        self.rewards = np.zeros(nenvs)
        self.had_ended_episodes = np.zeros(nenvs, dtype=np.bool)
        self.episode_lengths = np.zeros(nenvs)
        self.reward_queues = [deque([], maxlen=running_mean_size)
                              for _ in range(nenvs)]
        self.rewards_step =0


    def should_write_summaries(self):
        """ Returns true if it's time to write summaries. """
        return np.all(self.had_ended_episodes)

    def add_summaries(self):
        """ Writes summaries. """
        if "step" not in torch_steps().keys(): self.step_var = 0
        else: self.step_var = torch_steps()["step"]

        writer.add_scalar(
            f"{self.prefix}/total_reward",
            np.mean([q[-1] for q in self.reward_queues]),
            global_step  = self.step_var)
        writer.add_scalar(
            f"{self.prefix}/reward_mean_{self.reward_queues[0].maxlen}",
            np.mean([np.mean(q) for q in self.reward_queues]),
            global_step  = self.step_var)
        writer.add_scalar(
            f"{self.prefix}/episode_length",
            np.mean(self.episode_lengths),
            global_step  = self.step_var)
        
        if self.had_ended_episodes.size > 1:
            writer.add_scalar(
                f"{self.prefix}/min_reward",
                min(q[-1] for q in self.reward_queues),
                global_step  = self.step_var)
            writer.add_scalar(
                f"{self.prefix}/max_reward",
                max(q[-1] for q in self.reward_queues),
                global_step  = self.step_var)
        self.episode_lengths.fill(0)
        self.had_ended_episodes.fill(False)

    def step(self, action):
        self.step_count +=1
        clipped_action = np.clip(action.tolist(), self.env.action_space.low, self.env.action_space.high)
        obs, rew, done, info = self.env.step(action)
        self.rewards += rew
        self.episode_lengths[~self.had_ended_episodes] += 1

        info_collection = [info] if isinstance(info, dict) else info
        done_collection = [done] if isinstance(done, bool) else done
        done_indices = [i for i, info in enumerate(info_collection)
                        if info.get("real_done", done_collection[i])]
        for i in done_indices:
            if not self.had_ended_episodes[i]:
                self.had_ended_episodes[i] = True
            self.reward_queues[i].append(self.rewards[i])
            self.rewards[i] = 0

        if self.should_write_summaries():
            self.add_summaries()
        return obs, rew, done, info

    def reset(self, **kwargs):
        self.rewards_step =0
        self.rewards.fill(0)
        self.episode_lengths.fill(0)
        self.had_ended_episodes.fill(False)
        return self.env.reset(**kwargs)

The normalization wrapper will subtract running mean from observations and rewards and divide 
the resulting quantities by the  running variances.

In [7]:
from mujoco_wrappers import Normalize

env = Normalize(Summaries(gym.make("HalfCheetahMuJoCoEnv-v0")));
env.unwrapped.seed(0);

WalkerBase::__init__


/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Next, you will need to define a model for training. We suggest that you use two separate networks: one for policy
and another for value function. Each network should be a 3-layer MLP with 64 hidden units, $\mathrm{tanh}$ 
activation function, kernel matrices initialized with orthogonal initializer with parameter $\sqrt{2}$
and biases initialized with zeros. 

Our policy distribution is going to be multivariate normal with diagonal covariance. 
The network from above will predict the mean, and the covariance should be represented by a single 
(learned) vector of size 6 (corresponding to the dimensionality of the action space from above). 
You should initialize this vector to zero and take the exponent of it to always
have a non-negative quantity. 

Overall the model should return three things: predicted mean of the distribution, variance vector, 
value function. 

In [23]:
import torch.nn as nn
import torch.nn.functional as F


class Model(nn.Module):
    def __init__(self, 
                 input_dim = np.prod(np.array(env.observation_space.shape), axis = 0),
                 n_actions = np.array(env.action_space.shape),
                 hidden_dim = 64):
        super(type(self), self).__init__()
        self.input_layer = self.init_layer(nn.Linear(input_dim, hidden_dim))
        self.hidden_layer = self.init_layer(nn.Linear(hidden_dim, hidden_dim))

        self.policy = self.init_layer(nn.Linear(hidden_dim, n_actions))
        self.value_f = self.init_layer(nn.Linear(hidden_dim, 1))

        self.covariance = nn.Parameter(torch.zeros(1, n_actions))

    def init_layer(self, layer: torch.nn.Module, std = np.sqrt(2), bias_ = 0.):
        torch.nn.init.orthogonal_(layer.weight, std)
        if isinstance(layer, torch.nn.Linear): torch.nn.init.constant_(layer.bias, bias_)
        return layer 

    def policy_net(self, x):
        out = torch.tanh(self.input_layer(x))
        out = torch.tanh(self.hidden_layer(out))
        out = self.policy(out)
        return out, self.covariance.expand_as(out)
    
    def V_net(self, x):
        out = torch.tanh(self.input_layer(x))
        out = torch.tanh(self.hidden_layer(out))
        out = self.value_f(out)
        return out



This model will be wrapped by a `Policy`. The policy can work in two modes, but in either case 
it is going to return dictionary with string-type keys. The first mode is when the policy is 
used to sample actions for a trajectory which will later be used for training. In this case 
the flag `training` passed to `act` method is `False` and the method should return 
a `dict` with the following keys: 

* `"actions"`: actions to pass to the environment
* `"log_probs"`: log-probabilities of sampled actions
* `"values"`: value function $V^\pi(s)$ predictions.

We don't need to use the values under these keys for training, so all of them should be of type `np.ndarray`.

When `training` is `True`, the model is training on a given batch of observations. In this
case it should return a `dict` with the following keys

* `"distribution"`: an instance of multivariate normal distribution (`torch.distributions.MultivariateNormal` or `tf.distributions.MultivariateNormalDiag`)
* `"values"`: value function $V^\pi(s)$ prediction.

The distinction about the modes comes into play depending on where the policy is used: if it is called from `EnvRunner`, 
the `training` flag is `False`, if it is called from `PPO`, the `training` flag is `True`. These classed 
will be described below. 

In [31]:
from torch.distributions import MultivariateNormal, Normal

class Policy:
  def __init__(self, model):
      self.model = model
  def act(self, inputs, training=False) -> dict:
      if training:
          inputs = torch.Tensor(inputs).to(device)
          mean, logstd = self.model.policy_net(inputs)
          values = self.model.V_net(inputs)
          return {"distribution": Normal(mean, torch.exp(logstd)),
                  "values": values}
      else: 
          with torch.no_grad():
              inputs = torch.Tensor(inputs).unsqueeze(0).to(device) #one obs batch
              values = self.model.V_net(inputs)
              mean, logstd = self.model.policy_net(inputs)
              distribution = Normal(mean, torch.exp(logstd))

          return {"actions": distribution.sample()[0].detach().cpu().numpy(),
                  "log_probs":distribution.log_prob(distribution.sample()[0]).sum(dim = 1).view(-1).cpu().detach().numpy(),
                  "values": values.view(-1).cpu().detach().cpu().numpy()}


We will use `EnvRunner` to perform interactions with an environment with a policy for a fixed number of timesteps. Calling `.get_next()` on a runner will return a trajectory &mdash; dictionary 
containing keys

* `"observations"`
* `"rewards"` 
* `"resets"`
* `"actions"`
* all other keys that you defined in `Policy`,

under each of these keys there is a `np.ndarray` of specified length $T$ &mdash; the size of partial trajectory. 

Additionally, before returning a trajectory this runner can apply a list of transformations. 
Each transformation is simply a callable that should modify passed trajectory in-place.

In [10]:
class AsArray:
  """ 
  Converts lists of interactions to ndarray.
  """
  def __call__(self, trajectory):
    # Modify trajectory inplace. 
    for k, v in filter(lambda kv: kv[0] != "state",
                       trajectory.items()):
      trajectory[k] = np.asarray(v)

In [11]:
import numpy as np
from runners import EnvRunner

class DummyPolicy:
  def act(self, inputs, training=False):
    assert not training
    return {"actions": np.random.randn(6), "values": np.nan}
  
runner = EnvRunner(env, DummyPolicy(), 3,
                   transforms=[AsArray()])
trajectory = runner.get_next()

{k: v.shape for k, v in trajectory.items() if k != "state"}

{'actions': (3, 6),
 'observations': (3, 17),
 'resets': (3,),
 'rewards': (3,),
 'values': (3,)}

You will need to implement the following two transformations. 

The first is `GAE` that implements [Generalized Advantage Estimator](https://arxiv.org/abs/1506.02438).
In it you should add two keys to the trajectory: `"advantages"` and `"value_targets"`. In GAE the advantages
$A_t^{\mathrm{GAE}(\gamma,\lambda)}$ are essentially defined as the exponential 
moving average with parameter $\lambda$ of the regular advantages 
$\hat{A}^{(T)}(s_t) = \sum_{l=0}^{T-1-t} \gamma^l r_{t+l} + \gamma^{T} V^\pi(s_{T}) - V^\pi(s_t)$. 
The exact formula for the computation is the following

$$
A_{t}^{\mathrm{GAE}(\gamma,\lambda)} = \sum_{l=0}^{T-1-t} (\gamma\lambda)^l\delta_{t + l}^V, \, t \in [0, T)
$$
where $\delta_{t+l}^V = r_{t+l} + \gamma V^\pi(s_{t+l+1}) - V^\pi(s_{t+l})$. You can look at the 
derivation (formulas 11-16) in the paper. Don't forget to reset the summation on terminal
states as determined by the flags `trajectory["resets"]`. You can use `trajectory["values"]`
to get values of all observations except the most recent which is stored under 
 `trajectory["state"]["latest_observation"]`. For this observation you will need to call the policy 
 to get the value prediction.

Once you computed the advantages, you can get the targets for training the value function by adding 
back values:
$$
\hat{V}(s_{t+l}) = A_{t+l}^{\mathrm{GAE}(\gamma,\lambda)} + V(s_{t + l}),
$$
where $\hat{V}$ is a tensor of value targets that are used to train the value function. 

In [12]:
class GAE:
  """ Generalized Advantage Estimator. """
  def __init__(self, policy, gamma=0.99, lambda_=0.95):
    self.policy = policy
    self.gamma = gamma
    self.lambda_ = lambda_
    
  def __call__(self, trajectory):
      advantages = deque()
      next_values = deque()

      rewards = trajectory["rewards"]
      values = list(trajectory["values"])
      resets = 1 - trajectory["resets"] 
      acted_policy = self.policy.act(trajectory["state"]["latest_observation"], training = False)
      values.append(acted_policy["values"])

      prev_gae = 0
      for step in reversed(range(len(rewards))):
          TD_delta = rewards[step] + self.gamma * values[step + 1] * resets[step] - values[step] 
          advantage = prev_gae =  TD_delta + self.gamma * self.lambda_ * prev_gae * resets[step]
          advantages.appendleft(advantage)
          next_values.appendleft(advantage + values[step])
      
      trajectory['advantages'] = np.array(advantages)
      trajectory["value_targets"] = np.array(advantages)
      values = np.array(values)



The main advantage of PPO over simpler policy based methods like A2C is that it is possible
to train on the same trajectory for multiple gradient steps. The following class wraps 
an `EnvRunner`. It should call the runner to get a trajectory, then return minibatches 
from it for a number of epochs, shuffling the data before each epoch.

In [13]:
class TrajectorySampler:
  """ Samples minibatches from trajectory for a number of epochs. """
  def __init__(self, runner, num_epochs, num_minibatches, transforms=None):
    self.runner = runner
    self.num_epochs = num_epochs
    self.num_minibatches = num_minibatches
    self.transforms = transforms or []
    self.minibatch_count = 0
    self.epoch_count = 0
    self.trajectory = None

    self.elements_per_minibatch = self.num_epochs // self.num_minibatches
    self.indicies = []
    
  def shuffle_trajectory(self):
    """ Shuffles all elements in trajectory.
    
    Should be called at the beginning of each epoch.
    """
    self.indices  = np.arange(len(self.trajectory["values"]))
    np.random.shuffle(self.indices)
    for k,v in self.trajectory.items():
        if k != "state" : self.trajectory[k] = np.array(v)[self.indices.astype(int)]
    
  def get_next(self):
    """ Returns next minibatch.  """
    trajectory_minibatch = dict()
    if self.epoch_count > self.num_epochs: return trajectory_minibatch
    if self.epoch_count ==0 and self.minibatch_count == 0:
        
        self.trajectory = self.runner.get_next()
        self.shuffle_trajectory()
        start = 0
        end = start + self.minibatchsize-1       
        
    elif self.minibatch_count == self.num_minibatches:
        
        self.minibatch_count=0
        self.epoch_count += 1
        self.trajectory = self.runner.get_next()
        self.shuffle_trajectory()
        start = 0
        end = start + self.minibatchsize-1
    else:
        
        start = self.minibatch_count*self.minibatchsize
        end = start + self.minibatchsize-1
    
    for k,v in self.trajectory.items():
        if k != "state": 
            trajectory_minibatch[k] = v[self.indices [start:end]]
    
    self.minibatch_count+=1
    
    return trajectory_minibatch

A common trick to use with GAE is to normalize advantages, the following transformation does that. 

In [14]:
class NormalizeAdvantages:
  """ Normalizes advantages to have zero mean and variance 1. """
  def __call__(self, trajectory):
    adv = trajectory["advantages"]
    adv = (adv - adv.mean()) / (adv.std() + 1e-8)
    trajectory["advantages"] = adv

Finally, we can create our PPO runner. 

In [15]:
def make_ppo_runner(env, policy, num_runner_steps=2048,
                    gamma=0.99, lambda_=0.95, 
                    num_epochs=10, num_minibatches=32):
  """ Creates runner for PPO algorithm. """
  runner_transforms = [AsArray(),
                       GAE(policy, gamma=gamma, lambda_=lambda_)]
  runner = EnvRunner(env, policy, num_runner_steps, 
                     transforms=runner_transforms)
  
  sampler_transforms = [NormalizeAdvantages()]
  sampler = TrajectorySampler(runner, num_epochs=num_epochs, 
                              num_minibatches=num_minibatches,
                              transforms=sampler_transforms)
  return sampler

In the next cell you will need to implement Proximal Policy Optimization algorithm itself. The algorithm
modifies the typical policy gradient loss in the following way:

$$
J_{\pi}(s, a) = \frac{\pi_\theta(a|s)}{\pi_\theta^{\text{old}}(a|s)} \cdot A^{\mathrm{GAE}(\gamma,\lambda)}(s, a)
$$

$$
J_{\pi}^{\text{clipped}}(s, a) = \mathrm{clip}\left(
\frac{\pi_\theta(a|s)}{\pi_{\theta^{\text{old}}}(a|s)},
1 - \text{cliprange}, 1 + \text{cliprange}\right)\cdot A^{\mathrm{GAE(\gamma, \lambda)}}(s)\\
$$

$$
L_{\text{policy}} = -\frac{1}{T}\sum_{l=0}^{T-1}\min\left(J_\pi(s_{t + l}, a_{t + l}), J_{\pi}^{\text{clipped}}(s_{t + l}, a_{t + l})\right).
$$

The value loss is also modified:

$$
L_{V}^{\text{clipped}} = \frac{1}{T}\sum_{l=0}^{T-1} \max(l^{simple}(s_{t + l}), l^{clipped}(s_{t + l}))
$$
, where $l^{simple}$ is your standard critic loss
$$
l^{simple}(s_{t + l}) = [V_\theta(s_{t+l}) - G(s_{t + l})]^2
$$

and $l^{clipped}$ is a clipped version that limits large changes of the value function:
$$
l^{clipped}(s_{t + l}) = [
V_{\theta^{\text{old}}}(s_{t+l}) +
\text{clip}\left(
V_\theta(s_{t+l}) - V_{\theta^\text{old}}(s_{t+l}),
-\text{cliprange}, \text{cliprange}
\right) - G(s_{t + l})] ^ 2
$$

In [35]:
class PPO:
    def __init__(self, policy, optimizer,
                cliprange = 0.2,
                value_loss_coef = 0.25,
                max_grad_norm = 0.5):
      self.policy = policy
      self.optimizer = optimizer
      self.cliprange = cliprange
      self.value_loss_coef = value_loss_coef
      # Note that we don't need entropy regularization for this env.
      self.max_grad_norm = max_grad_norm
      self._policy_loss = None
      self._value_loss = None
      self.total_loss = None
      
    def policy_loss(self, trajectory, act):
        """ Computes and returns policy loss on a given trajectory. """
        
        actions = torch.tensor(trajectory["actions"]).to(device) 
        old_log_probs = torch.tensor(trajectory["log_probs"]).to(device).flatten() 
        new_logprobs = act["distribution"].log_prob(actions).sum(1)

        
        ratio = torch.exp(new_logprobs - old_log_probs)
        self._policy_loss = torch.mean(torch.max(- ratio * torch.Tensor(trajectory["advantages"]).to(device), 
                                                - torch.clamp(ratio, 1 - self.cliprange, 1 + self.cliprange) * torch.Tensor(trajectory["advantages"]).to(device) ))
        
        return self._policy_loss
          
        
    def value_loss(self, trajectory, act):
        """ Computes and returns value loss on a given trajectory. """
        new_values = act["values"].flatten() 
        returns = torch.tensor(trajectory["value_targets"]).to(device)
        values = torch.tensor(trajectory["values"]).to(device).flatten()

        v_loss1  =(returns - new_values).pow(2)
        clipped_values = values + torch.clamp(new_values - values,
                                              -self.cliprange,
                                              self.cliprange)
        v_loss2 = (clipped_values - returns ).pow(2)
        
        self._value_loss = 0.5 * (torch.max(v_loss1,v_loss2)).mean()
        return self._value_loss

    def loss(self, trajectory):
        act = self.policy.act(trajectory["observations"], training=True)
        policy_loss = self.policy_loss(trajectory, act)
        value_loss = self.value_loss(trajectory, act)
        total_loss = policy_loss + self.value_loss_coef * value_loss
        self.total_loss = total_loss # just for monitoring
        return total_loss

    def get_info(self):
          return {'policy_loss': self._policy_loss,
                  'value_loss': self._value_loss,
                  'total_loss': self.total_loss}
        
    def step(self, trajectory):
        """ Computes the loss function and performs a single gradient step. """
        loss = self.loss(trajectory)
        optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(self.policy.model.parameters(), self.max_grad_norm)
        optimizer.step()

Now everything is ready to do training. In one million of interactions it should be possible to 
achieve the total raw reward of about 1500. You should plot this quantity with respect to 
`runner.step_var` &mdash; the number of interactions with the environment. It is highly 
encouraged to also provide plots of the following quantities (these are useful for debugging as well):

* [Coefficient of Determination](https://en.wikipedia.org/wiki/Coefficient_of_determination) between 
value targets and value predictions
* Entropy of the policy $\pi$
* Value loss
* Policy loss
* Value targets
* Value predictions
* Gradient norm
* Advantages

For optimization it is suggested to use Adam optimizer with linearly annealing learning rate 
from 3e-4 to 0 and epsilon 1e-5.

In [39]:
hidden_units =64
input_shape = np.array(env.observation_space.shape).prod()
n_actions = np.prod(env.action_space.shape)

lr = lambda f: f * learning_rate
anneal_lr = True

learning_rate = 3e-4
cliprange=0.2
value_loss_coef=0.25
max_grad_norm=0.5

nenvs =1
nsteps = num_runner_steps=2048 
num_batch = nsteps * nenvs

gamma=0.99
lambda_=0.95
num_epochs=100
num_minibatches=32

total_timesteps = 5000
train_mini_batch = num_batch//num_minibatches
num_updates = total_timesteps//num_minibatches

print(f"#Steps = {total_timesteps}, minibatchsize = {train_mini_batch}, #Updates = {num_updates}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ppo_agent = Model(input_shape,n_actions).to(device)
policy = Policy(ppo_agent)
optimizer = torch.optim.Adam(policy.model.parameters(),lr = learning_rate, eps = 1e-5 )
ppo = PPO(policy, optimizer, cliprange = cliprange,
                             value_loss_coef = value_loss_coef,
                             max_grad_norm = max_grad_norm)


minibatcher = make_ppo_runner(env, policy, num_runner_steps,
                              gamma, lambda_, 
                              num_epochs, num_minibatches)

minibatcher.minibatchsize = train_mini_batch

#Steps = 5000, minibatchsize = 64, #Updates = 156


In [ ]:
from tqdm import tqdm
for update in tqdm(range(1, int(num_updates//10) + 1), total = int(num_updates//10) ):
    minibatcher.epoch_count = 0
    minibatcher.minibatch_count = 0
    total_loss =[]
    value_loss =[]
    policy_loss =[]
    if anneal_lr:
        frac = 1.0 - (update - 1.0) / num_updates
        lrnow = lr(frac)
        ppo.optimizer.param_groups[0]['lr'] = lrnow
    for _ in range (num_epochs):
        for _ in range(num_minibatches):
            trajectory_mbatch = minibatcher.get_next()
            ppo.step(trajectory_mbatch)
            step = optimizer.state[optimizer.param_groups[0]["params"][-1]]["step"]

            total_loss.append(ppo.total_loss.detach().cpu().numpy())
            value_loss.append(ppo._value_loss.detach().cpu().numpy())
            policy_loss.append(ppo._policy_loss.detach().cpu().numpy())
        
        writer.add_scalar(
            f"{env.spec.id}/total_loss",
            np.mean(total_loss),
            global_step=step)
        writer.add_scalar(
            f"{env.spec.id}/value_loss",
            np.mean(value_loss),
            global_step=step)
        writer.add_scalar(
            f"{env.spec.id}/policy_loss",
            np.mean(policy_loss),
            global_step=step)

  0%|          | 0/15 [00:00<?, ?it/s]

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs